In [ ]:
import os
import xarray as xr
import visualize as viz
import holoviews as hv

hv.extension('bokeh')

def get_map(diag_type, diags, label=None):
    global_avg = hv.HoloMap(kdims=['field'])
    for variable in diags:
        if variable.endswith(diag_type) and set(diags[variable].dims) == {"time"}:
            long_name  = diags[variable].attrs.get('long_name', variable)
            global_avg[long_name] = (
                hv.Curve(diags[variable], label=label)
                .redim.unit(variable=diags[variable].attrs.get('units', ''))
            )

    return global_avg


def convert_time_index_to_datetime(ds, dim):
    return ds.assign_coords({dim: ds.indexes[dim].to_datetimeindex()})

In [ ]:

diags = xr.open_dataset('diags.nc')

# convert to datetime for plotting with bokeh
# cftime time indices raise tons of annoying errors with bokeh
diags = convert_time_index_to_datetime(diags, 'time')

# Information about diagnostics

In [ ]:
diags.info()

# Maps

In [ ]:
viz.plot_cube(viz.mappable_var(diags, 'pwat_run_initial'));

In [ ]:
viz.plot_cube(viz.mappable_var(diags, 'pwat_run_final'));

In [ ]:
viz.plot_cube(viz.mappable_var(diags, 'pwat_verification_final'));

In [ ]:
%%output size=100
%%opts Curve[aspect=1.61] {+framewise}
get_map('rms_global', diags, label='RMSE Global')

In [ ]:
%%opts Curve[aspect=1.61] {+framewise}
get_map('global_avg', diags, label='Global Average')